In [1]:
!nvidia-smi

Sat Jul  1 12:10:24 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [12]:
!pip install -q transformers einops accelerate langchain bitsandbytes

In [15]:
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer, pipeline
import torch

In [16]:
model = "tiiuae/falcon-7b-instruct" #tiiuae/falcon-40b-instruct

tokenizer = AutoTokenizer.from_pretrained(model)

pipeline = pipeline(
    "text-generation", #task
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    max_length=200,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    pad_token_id=tokenizer.eos_token_id
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.
The model 'RWForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusFor

In [17]:
llm = HuggingFacePipeline(pipeline = pipeline, model_kwargs = {'temperature':0})

In [19]:
# testing the model
llm("What is the capital of India")

'?\nThe capital of India is New Delhi.'

In [20]:
from langchain import PromptTemplate,  LLMChain

template = """
You are an intelligent chatbot. Help the following question with brilliant answers.
Question: {question}
Answer:"""
prompt = PromptTemplate(template=template, input_variables=["question"])

llm_chain = LLMChain(prompt=prompt, llm=llm)

In [ ]:
question = "Explain what is Artificial Intellience as Nursery Rhymes "

print(llm_chain.run(question))

In [21]:
question = "What is the capital of India"

output = llm_chain.run(question)
output = output.split('\n')[0].strip()
print(output)

New Delhi


In [23]:
question = "Who is the Prime Minister of India"

output = llm_chain.run(question)
output = output.split('\n')[0].strip()
print(output)

The current Prime Minister of India is Narendra Modi. He assumed office on 26 July 2016.


In [25]:
# function for output generation
def question_answer_func(question:str)->str:
  output = llm_chain.run(question)
  output = output.split('\n')[0].strip()
  return output

In [32]:
question = input("Enter the quesition: ")
question_answer_func(question)

Enter the quesition: What is the Capital of India


'New Delhi'

In [33]:
question = input("Enter the quesition: ")
question_answer_func(question)

Enter the quesition: Who is Elon Musk


"Elon Musk is a busy guy. Aside from his day job as an entrepreneur and engineer, he's also a SpaceX founder, Tesla CEO, and an advocate for renewable energy. He's involved in several projects and has even been working on a new communication system for space travel. He's an ambitious man with an impressive track record."

In [34]:
question = input("Enter the quesition: ")
question_answer_func(question)

Enter the quesition: Who is Andrej Karpathy


'Andrej Karpathy is a well-known deep learning researcher and a former employee of Google. He is known for developing neural networks that can learn to do things like recognize objects in images and generate text. He has written papers and tutorials on deep learning that have been used by thousands of people around the world to learn about the field.'

In [35]:
question = input("Enter the quesition: ")
question_answer_func(question)

Enter the quesition: What is self-attention mechanism in transformers


'Self-attention mechanism in transformers is a key technique in deep learning that helps in capturing complex patterns from the input sequence, allowing a model to learn important features for the task it is being trained on.'